In [50]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

In [51]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [52]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [53]:
ticket_prices = {
    "mumbai": "$799", "berlin": "$899", "delhi": "$1400", "chennai": "$499"
}


def get_ticket_price(destination_city):
    print(f"Tool get ticket price for {destination_city}")
    city = destination_city.lower()  # Convert input to lowercase
    return ticket_prices.get(city, "Unknown")

In [54]:
print(get_ticket_price("delhi"))

Tool get ticket price for delhi
$1400


In [66]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that customer want to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    }
}

In [67]:
tools = [{"type": "function", "function": price_function}]

In [73]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [72]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id":tool_call.id
    }
    return response, city

In [74]:
gr.ChatInterface(fn=chat).launch()

D:\Artificial\.venv\Lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.
